<a href="https://colab.research.google.com/github/georgezefko/Anomaly-Detection-for-Sequential-Data/blob/main/VAE_GRU_Re-encoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [14]:
!pip3 install pickle5

In [15]:
%matplotlib inline
%config InlineBackend.figure_format='retina'
import numpy as np
import torch
from torch import nn, optim
from torch import distributions
from torch.utils.data import DataLoader, TensorDataset

from torch.autograd import Variable
import os
import numpy as np
from random import randint
import os
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
import copy
from torch.nn.parameter import Parameter
import math
from torch.nn import functional as F
import seaborn as sns
from sklearn.metrics import confusion_matrix, precision_recall_curve, roc_curve, auc, confusion_matrix, roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

import pickle5 as pickle
import statsmodels.api
import os
from tqdm import tqdm
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from pandas.plotting import register_matplotlib_converters


sns.set(style='whitegrid', palette='muted', font_scale=1.2)

HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#93D30C", "#8F00FF"]

sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))

rcParams['figure.figsize'] = 14, 10
register_matplotlib_converters()

batch_size = 64
green_mobility = True
max=False


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
cuda = torch.cuda.is_available()

In [16]:
def heartbeat(data_train,data_test_val):
  data = np.concatenate((data_train, data_test_val), axis=0)
  df = pd.DataFrame(data)
  new_columns = list(df.columns)
  new_columns[0] = 'target'
  df.columns = new_columns
  df.target = [int(i) for i in df.target]

  normal_df = df[df.target == 1].drop(labels='target', axis=1)
  anomaly_df = df[df.target != 1].drop(labels='target', axis=1)
  return normal_df,anomaly_df

In [17]:
from scipy.signal import butter,filtfilt
# Filter requirements.
T = 5.0 # Sample Period
fs = 20.0 # sample rate, Hz
cutoff = 1.2 # desired cutoff frequency of the filter, Hz , slightly higher than actual 1.2 Hz
nyq = 0.5 * fs # Nyquist Frequency
order = 2 # sin wave can be approx represented as quadratic
n = int(T * fs) # total number of samples
 
def butter_lowpass_filter(data, cutoff, fs, order):
  normal_cutoff = cutoff / nyq
  # Get the filter coefficients 
  b, a = butter(order, normal_cutoff, btype='low', analog=False)
  y = filtfilt(b, a, data)
  return y
def smoothing(Normal,Anomaly): 
  Normal_smooth = pd.DataFrame()
  for i in range(Normal.shape[0]):
    Normal_smooth = Normal_smooth.append(pd.DataFrame(butter_lowpass_filter(Normal.loc[i].values, cutoff, fs, order)).transpose())
  Normal_smooth = Normal_smooth.reset_index(drop = True) 
 
  Anomaly_smooth = pd.DataFrame()
  for i in range(Anomaly.shape[0]):
    Anomaly_smooth = Anomaly_smooth.append(pd.DataFrame(butter_lowpass_filter(Anomaly.loc[i].values, cutoff, fs, order)).transpose())
  Anomaly_smooth = Anomaly_smooth.reset_index(drop = True)
  return Normal_smooth,Anomaly_smooth

In [18]:
if green_mobility:
  path = '/content/drive/MyDrive/AdvancedML/data/resampled_df.pickle'
  with open(path,'rb') as fh:
      data = pickle.load(fh)
  data = data[['GM.acc.xyz.z_resampled','IRI_mean']]
  data_unstacked = data.iloc[:,0].apply(list).apply(pd.Series)
  df = pd.concat([data_unstacked,data['IRI_mean']],axis=1)
  normal_df = df[df['IRI_mean'] <= 2].drop(labels='IRI_mean',axis=1).reset_index(drop = True)
  anomaly_df = df[df['IRI_mean'] > 2].drop(labels='IRI_mean',axis=1).reset_index(drop = True)
else:
  data_train = np.loadtxt('/content/drive/MyDrive/AdvancedML/data/ECG5000/ECG5000_TRAIN', delimiter=',')
  data_test_val = np.loadtxt('/content/drive/MyDrive/AdvancedML/data/ECG5000/ECG5000_TEST', delimiter=',')[:-1]
  normal_df,anomaly_df = heartbeat(data_train,data_test_val)

#for smoothing(Needs correction) it has been used only for experimentation
#normal_df,anomaly_df=smoothing(normal_df,anomaly_df)

RANDOM_SEED = 42
train_df, val_df = train_test_split(normal_df,test_size=0.33,random_state=RANDOM_SEED)
val_df, test_df = train_test_split(val_df,test_size=0.15, random_state=RANDOM_SEED)


# ## Scaling your dataset
if max:
  scaler = MinMaxScaler(feature_range=(0, 1))
  train_df = scaler.fit_transform(train_df)
  test_df = scaler.transform(test_df)
  val_df = scaler.transform(val_df)
  anomaly_df = scaler.fit_transform(anomaly_df)
else:
  scaler = StandardScaler()
  train_df = scaler.fit_transform(train_df)
  test_df = scaler.transform(test_df)
  val_df = scaler.transform(val_df)
  anomaly_df = scaler.fit_transform(anomaly_df)



train_df = np.expand_dims(train_df,-1)
test_df = np.expand_dims(test_df,-1)
val_df = np.expand_dims(val_df,-1)
anomaly_df = np.expand_dims(anomaly_df,-1)

seq_len = train_df.shape[1]
n_features = train_df.shape[2]

train_dataset = TensorDataset(torch.from_numpy(train_df).float())
test_dataset = TensorDataset(torch.from_numpy(test_df).float())
val_dataset = TensorDataset(torch.from_numpy(val_df).float())
anomaly_dataset = TensorDataset(torch.from_numpy(anomaly_df).float())



train_loader = DataLoader(train_dataset, batch_size=batch_size, pin_memory=cuda,drop_last=True)
test_loader  = DataLoader(test_dataset, batch_size=batch_size, pin_memory=cuda,drop_last=True)
val_loader  = DataLoader(val_dataset, batch_size=batch_size, pin_memory=cuda, drop_last=True)
anomaly_loader = DataLoader(anomaly_dataset, batch_size=batch_size, pin_memory=cuda,drop_last=True)

In [19]:
class Encoder(nn.Module):

  def __init__(self, n_features, hidden_size, hidden_layer_depth, latent_length, bidirectional,dropout):

    super(Encoder,self).__init__()


    self.n_features = n_features
    self.hidden_size = hidden_size
    self.hidden_layer_depth = hidden_layer_depth
    self.latent_length = latent_length
    self.bidirectional = bidirectional
    self.direction_factor = (2 if self.bidirectional else 1)
    


    self.rnn1 = nn.GRU( input_size=self.n_features, 
                       hidden_size=self.hidden_size, 
                       dropout = dropout, 
                       num_layers = self.hidden_layer_depth,
                       batch_first=True,bidirectional = self.bidirectional)
    
    
      
  def forward(self, x):
    
    x = x.to(device)
    batch = x.size(0)
    h0 = torch.zeros(hidden_layer_depth*self.direction_factor, batch, self.hidden_size).to(device)

    out,hidden_n = self.rnn1(x,h0) 
    if self.bidirectional:

      last_hidden_state = hidden_n.view(hidden_layer_depth, self.direction_factor, batch, self.hidden_size)[-1] #(num_directions, batch, hidden_size)
      h_1, h_2 = last_hidden_state[0], last_hidden_state[1]

      #X = h_1 + h_2                # Add both states (needs different input size for first linear layer)
      hidden = torch.cat((h_1, h_2), 1) # Concatenate both states
    else:
      hidden = hidden_n[-1, :, :]
    return hidden



In [20]:
class Latent(nn.Module):
 
    def __init__(self, hidden_size, latent_length,bidirectional):
        super(Latent, self).__init__()

        self.bidirectional = bidirectional
        self.direction_factor = (2 if self.bidirectional else 1)
        self.hidden_size = hidden_size
        self.latent_length = latent_length

        self.hidden_to_mean = nn.Linear(self.hidden_size *self.direction_factor, self.latent_length)
        self.hidden_to_logvar = nn.Linear(self.hidden_size * self.direction_factor, self.latent_length)

        nn.init.xavier_uniform_(self.hidden_to_mean.weight)
        nn.init.xavier_uniform_(self.hidden_to_logvar.weight)

    def forward(self, cell_output):
        
        self.latent_mean = self.hidden_to_mean(cell_output)
        self.latent_logvar = self.hidden_to_logvar(cell_output)
        z_mu = self.latent_mean
        z_var = self.latent_logvar
        

        
        std = torch.exp(0.5 * self.latent_logvar)
        eps = torch.randn_like(std)
        sample = eps.mul(std).add_(self.latent_mean)

        return sample, z_mu, z_var
        

In [21]:

class Decoder(nn.Module):

  def __init__(self, seq_len, batch_size, hidden_size, hidden_layer_depth, latent_length, n_features,bidirectional):
    super(Decoder, self).__init__()

    self.hidden_size = hidden_size
    self.batch_size = batch_size
    self.seq_len = seq_len
    self.hidden_layer_depth = hidden_layer_depth
    self.latent_length = latent_length
    self.n_features = n_features
    self.bidirectional = bidirectional
    self.direction_factor = (2 if self.bidirectional else 1)
    
    

    self.rnn1 = nn.GRU(input_size=1,
                       hidden_size=self.hidden_size,
                       dropout = dropout,
                       num_layers=hidden_layer_depth,
                       batch_first=True,
                       bidirectional= bidirectional
                       )

    

    self.latent_to_hidden = nn.Linear(self.latent_length, self.hidden_size)
    self.hidden_to_output = nn.Linear(self.hidden_size*self.direction_factor, self.n_features)

    self.decoder_inputs = torch.zeros( self.batch_size, self.seq_len, 1, requires_grad=True).to(device)

    nn.init.xavier_uniform_(self.latent_to_hidden.weight)
    nn.init.xavier_uniform_(self.hidden_to_output.weight)


  def forward(self, x):
    
    x = x.to(device)
    h_state = self.latent_to_hidden(x)
    
    h_0 = torch.stack([h_state for _ in range(self.hidden_layer_depth*self.direction_factor)]).to(device)
    
    decoder_output, _ = self.rnn1(self.decoder_inputs, h_0)
    

    out = self.hidden_to_output(decoder_output)

    return out

In [22]:
class VAE(nn.Module):

  def __init__(self, enc,lat,dec):

    super(VAE,self).__init__()

  
    self.enc = enc
    self.lat = lat
    self.dec = dec
    

  def forward(self, x):

    encoder_output = self.enc(x)
    sample,z_mu,z_var = self.lat(encoder_output)
    x_hat = self.dec(sample)
    re_encoder_output = self.enc(x_hat)
    rec_sample,rec_z,rec_var = self.lat(re_encoder_output)



    return x_hat, z_mu, z_var,sample, rec_sample,rec_z,rec_var

In [23]:
hidden_size = 256
hidden_layer_depth = 2
latent_length = 10
dropout = 0.5
bidirectional = True
# encoder
encoder = Encoder( n_features, hidden_size, hidden_layer_depth, latent_length,bidirectional, dropout)

#latent
latent = Latent( hidden_size, latent_length, bidirectional)

# decoder
decoder = Decoder(seq_len, batch_size, hidden_size, hidden_layer_depth, latent_length, n_features,bidirectional)

# vae
model = VAE(encoder,latent,decoder).to(device)



print(model)

VAE(
  (enc): Encoder(
    (rnn1): GRU(1, 256, num_layers=2, batch_first=True, dropout=0.5, bidirectional=True)
  )
  (lat): Latent(
    (hidden_to_mean): Linear(in_features=512, out_features=10, bias=True)
    (hidden_to_logvar): Linear(in_features=512, out_features=10, bias=True)
  )
  (dec): Decoder(
    (rnn1): GRU(1, 256, num_layers=2, batch_first=True, dropout=0.5, bidirectional=True)
    (latent_to_hidden): Linear(in_features=10, out_features=256, bias=True)
    (hidden_to_output): Linear(in_features=512, out_features=1, bias=True)
  )
)


In [24]:
def loss(X, x_hat, z_mu, z_var,sample, rec_sample, rec_z, rec_var, criterion):
  

  if criterion == 'MSELoss':
    criterion = nn.MSELoss(size_average=False,reduction='sum')
  elif criterion == 'SmoothL1Loss':
    criterion = nn.SmoothL1Loss(size_average=False,reduction='sum')

  recon_loss = criterion(x_hat, X)
  latent_recon_loss = criterion(sample,rec_sample)
  KL_loss = -0.5 * torch.mean(1 + z_var - z_mu**2 - torch.exp(z_var))
  KL_loss_latent = -0.5 * torch.mean(1 + rec_var - rec_z**2 - torch.exp(rec_var))
  elbo_loss = recon_loss +  KL_loss
  total_loss = recon_loss +  KL_loss + latent_recon_loss + KL_loss_latent

  return total_loss,elbo_loss, KL_loss,KL_loss_latent

In [25]:
optimizer = optim.Adam(model.parameters(), lr=0.0006796395231964973) # lr derived from optimazation method

#optimizer = optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)

In [26]:
#Test model
X= next(iter(train_loader))
x = X[0].to(device)
outputs = model(x.float())


x_hat, z_mu, z_var,sample,rec_sample,rec_z,rec_var = outputs

print('x',x.shape)
print('mu',z_mu.shape)
print('var',z_var.shape)
print('x_hat',x_hat.shape)
print('re_z',rec_z.shape)
print('rec_var',rec_var.shape)
print('rec_sample',rec_sample.shape)


x torch.Size([64, 369, 1])
mu torch.Size([64, 10])
var torch.Size([64, 10])
x_hat torch.Size([64, 369, 1])
re_z torch.Size([64, 10])
rec_var torch.Size([64, 10])
rec_sample torch.Size([64, 10])


In [27]:
#Test loss
total_loss, elbo_loss, KL_loss,KL_latent_loss = loss(x, x_hat, z_mu, z_var,sample, rec_sample, rec_z, rec_var,criterion='SmoothL1Loss')

print(elbo_loss)
print(total_loss)
print(KL_loss)
print(KL_latent_loss)

tensor(9512.1992, device='cuda:0', grad_fn=<AddBackward0>)
tensor(9932.0928, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.0056, device='cuda:0', grad_fn=<MulBackward0>)
tensor(0.0055, device='cuda:0', grad_fn=<MulBackward0>)


/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


In [28]:
def train():
 

  model.train()

  train_loss = []
  train_kl = []
  train_total = []
  train_latent = []
  
  

  for t, X in enumerate(train_loader):

    X =X[0]
    X = X.to(device)

    optimizer.zero_grad()

    x_hat, z_mu, z_var,sample, rec_sample,rec_z,rec_var = model(X)

    total_loss,elbo_loss, KL_loss,KL_loss_latent = loss(X, x_hat, z_mu, z_var,sample, rec_sample,rec_z,rec_var, criterion = 'MSELoss')

    elbo_loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm = 5)

    train_loss.append(elbo_loss.item())
    train_kl.append(KL_loss.item())
    train_total.append(total_loss.item())
    train_latent.append(KL_loss_latent.item())
    

    optimizer.step()
  
  train_elbo_loss = np.mean(train_loss)
  train_kl_loss = np.mean(train_kl)
  train_total_loss = np.mean(train_loss)
  train_kl_latent_loss = np.mean(train_latent)
 
 

  return  train_total_loss, train_elbo_loss, train_kl_loss, train_kl_latent_loss

In [29]:
def test():

  model.eval()

  with torch.no_grad():
    test_loss = []
    test_kl = []
    test_total = []
    test_latent = []
    
    for t, x in enumerate(val_loader):

      x = x[0]
      x = x.to(device)
      x_hat, z_mu, z_var,sample, rec_sample,rec_z, rec_var = model(x)
      total_loss,elbo_loss, KL_loss, KL_loss_latent = loss(x, x_hat, z_mu, z_var,sample, rec_sample,rec_z,rec_var, criterion = 'MSELoss')

      

      test_loss.append(elbo_loss.item())
      test_kl.append(KL_loss.item())
      test_total.append(total_loss.item())
      test_latent.append(KL_loss_latent.item())
      

  test_elbo_loss = np.mean(test_loss)
  test_kl_loss = np.mean(test_kl)
  test_total_loss = np.mean(test_total)
  test_kl_latent_loss = np.mean(test_latent)
  

  return model.eval(), test_total_loss, test_elbo_loss, test_kl_loss, test_kl_latent_loss

In [30]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
epochs = 100

best_model_wts = copy.deepcopy(model.state_dict())
best_loss = 10000.0
history_elbo = dict(train_elbo=[], test_elbo=[])
history_kl = dict(train_kl=[], test_kl=[])
history_total = dict(train_total=[], test_total=[])
history_latent = dict(train_latent=[], test_latent=[])


device = torch.device("cuda:0" if cuda else "cpu")
print("Using device:", device)

for e in range(epochs):

  start_time = time.time()


  train_total_loss, train_elbo_loss, train_kl_loss, train_kl_latent_loss = train()
  model, test_total_loss, test_elbo_loss, test_kl_loss, test_kl_latent_loss = test()

  end_time = time.time()

  epoch_mins, epoch_secs = epoch_time(start_time, end_time)

  if best_loss > test_elbo_loss:
    best_test_loss = test_elbo_loss
    best_model_wts = copy.deepcopy(model.state_dict())

  print(f'Epoch: {e+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
  print(f'\t train total loss {train_total_loss} |train elbo loss {train_elbo_loss}| train kl loss {train_kl_loss} | train kl_lat loss {train_kl_latent_loss}') 
  print(f'\t val total loss {test_total_loss}    | val elbo loss {test_elbo_loss}  | val kl loss {test_kl_loss}    |val kl_lat loss {test_kl_latent_loss}') 


  history_elbo['train_elbo'].append(train_elbo_loss)
  history_elbo['test_elbo'].append(test_elbo_loss)

  history_total['train_total'].append(train_total_loss)
  history_total['test_total'].append(test_total_loss)

  history_kl['train_kl'].append(train_kl_loss)
  history_kl['test_kl'].append(test_kl_loss)

  history_latent['train_latent'].append(train_kl_latent_loss)
  history_latent['test_latent'].append(test_kl_latent_loss)


In [32]:
# save the model
MODEL_PATH = 'model.pth'
torch.save(model, MODEL_PATH)

In [ ]:
fig = plt.figure(figsize=(20,15))


fig.add_subplot(2,2,1)
plt.plot(list(range(0,len(history_elbo['train_elbo']))), history_elbo['train_elbo'],'r')
plt.plot(list(range(0,len(history_elbo['test_elbo']))), history_elbo['test_elbo'],'g')
plt.legend(['Train ELBO','Val ELBO'])
plt.xlabel('EPOCH')
plt.ylabel('Loss')
plt.title('ELBO Loss')


fig.add_subplot(2,2,2)
plt.plot(list(range(0,len(history_total['train_total']))), history_total['train_total'], 'r')
plt.plot(list(range(0,len(history_total['test_total']))), history_total['test_total'],'g')
plt.legend(['Train Total Loss','Validation Total Loss'])
plt.xlabel('EPOCH')
plt.ylabel('Loss')
plt.title('Total Loss')




fig.add_subplot(2,2, 3)
plt.plot(list(range(0,len(history_kl['train_kl']))), history_kl['train_kl'], 'r')
plt.plot(list(range(0,len(history_kl['test_kl']))), history_kl['test_kl'],'g')
plt.legend(['Train KL','Val KL'])
plt.xlabel('EPOCH')
plt.ylabel('Loss')
plt.title('KL Loss')


fig.add_subplot(2,2, 4)
plt.plot(list(range(0,len(history_latent['train_latent']))), history_latent['train_latent'], 'r')
plt.plot(list(range(0,len(history_latent['test_latent']))), history_latent['test_latent'],'g')
plt.legend(['Train R-KL','Val R-KL'])
plt.xlabel('EPOCH')
plt.ylabel('Loss')
plt.title('KL Loss Re-encoder')




#plt.savefig('/content/drive/MyDrive/AdvancedML/data/ecg_losses_vraebigru.png')
plt.show();



In [34]:
def predict(model, dataset):
    predictions = []
    enc_mean = []
    rec_mean = []
    with torch.no_grad():
        model = model.eval()
    for t, X in enumerate(dataset):
        X = X[0]
        x = X.to(device)
        x_hat, z_mu, z_var,sample, rec_sample,rec_z, rec_var = model(x)

        x_hat = x_hat.cpu().detach().numpy()
        z_mu = z_mu.cpu().detach().numpy()
        rec_z = rec_z.cpu().detach().numpy()

        x_hat = np.squeeze(x_hat, axis=2)
        predictions.append(x_hat)
        enc_mean.append(z_mu)
        rec_mean.append(rec_z)
       
    predictions = np.concatenate(predictions, axis=0, out= None)
    enc_mean = np.concatenate(enc_mean, axis=0, out= None)
    rec_mean = np.concatenate(rec_mean, axis=0, out= None)
    return predictions,enc_mean,rec_mean

In [35]:
predict_train,mu,re_mu = predict(model, train_loader)
predict_anomaly, mu_anomaly, re_mu_anomaly= predict(model, anomaly_loader)

In [36]:
from sklearn.manifold import TSNE

def projections(normal,anomaly):
  mn = pd.DataFrame(normal)
  mn['label']='Normal'
  ma = pd.DataFrame(anomaly)
  ma['label']='Anomaly'
  m = mn.append(ma)


  data_subset = m.iloc[:,0:10].values
  time_start = time.time()
  tsne = TSNE(n_components=2, verbose=1, perplexity=20, n_iter=1000)
  tsne_results = tsne.fit_transform(data_subset)
  print('t-SNE done! Time elapsed: {} seconds'.format(time.time()-time_start))

  m['Projection-1'] = tsne_results[:,0]
  m['Projection-2'] = tsne_results[:,1]
  plt.figure(figsize=(16,10))
  sns.scatterplot(
    x='Projection-1', y='Projection-2',
    hue="label",
    palette=sns.color_palette("hls", 2),
    data=m,
    legend="full",
    alpha=0.9)
  #plt.savefig('/content/drive/MyDrive/AdvancedML/data/ecg_encoder_latent.png')
  plt.show();

In [ ]:
#Projections encoder
projections(mu,mu_anomaly)
# Projections re-encoder
projections(re_mu,re_mu_anomaly)

In [40]:
def lillefor(data,predict,test=True):
  if test:
  #Lillefors test on test normal
    residual = (data - predict)
    ksstat = np.zeros(residual.shape[0])
    for i in range(residual.shape[0]):
     ksstat[i] = statsmodels.stats.diagnostic.lilliefors(residual[i,], dist='norm', pvalmethod='table')[0]
    sns.distplot(ksstat, bins=40, kde=True)
    #plt.savefig('/content/drive/MyDrive/AdvancedML/data/bigru_testnormal_rmse.png')
    plt.show()
    return ksstat
  else:
    #MSE on test normal
    sq_loss = np.mean(( data- predict)**2, axis=1)
    sns.distplot(sq_loss, bins=40, kde=True)
    plt.show()
    return sq_loss

In [ ]:
train_df = train_df[:predict_train.shape[0]]
train_original = np.squeeze(train_df,axis=2)
train_recon = lillefor(train_original,predict_train)

In [ ]:
predict_test,_,_ = predict(model, test_loader)
test_df=test_df[:predict_test.shape[0]]
test_original = np.squeeze(test_df,axis=2)
test_recon = lillefor(test_original,predict_test)

In [ ]:
anomaly_df = anomaly_df[:predict_anomaly.shape[0]]
anomaly_original = np.squeeze(anomaly_df)
anomaly_recon = lillefor(anomaly_original,predict_anomaly)

In [44]:
def plot_prediction(data, reconstructed, title, ax):
  ax.plot(data,'r', label='true')
  ax.plot(reconstructed, 'b',label='reconstructed')
  ax.set_title(f'{title} (Total loss: {np.around(np.mean(( data- reconstructed)**2), 2)})')
  ax.legend()

In [ ]:
fig, axs = plt.subplots(nrows=2,ncols=6,sharey=True,sharex=True,figsize=(22, 8))

for i, data in enumerate(test_original[:6]):
  plot_prediction(data, predict_test[i], title='Normal', ax=axs[0, i])

for i, data in enumerate(anomaly_original[:6]):
  plot_prediction(data, predict_anomaly[i], title='Anomaly', ax=axs[1, i])

fig.tight_layout();
#plt.savefig('/content/drive/MyDrive/AdvancedML/data/green_recon_smooth_vraegruuni.png')

In [ ]:
label = np.concatenate((np.zeros(len(test_recon)),np.ones(len(anomaly_recon))))
#total_losses = np.concatenate((train_sq_loss,anomaly_sq_loss),axis=0)
total_recon = np.concatenate((test_recon,anomaly_recon),axis=0)
error_df = pd.DataFrame({'Reconstruction_error': total_recon,'Label':label})
#error_df = error_df.sample(frac=1)
error_df.head(2)

In [ ]:
false_pos_rate, true_pos_rate, thresholds = roc_curve(error_df.Label, error_df.Reconstruction_error.values)
roc_auc = auc(false_pos_rate, true_pos_rate,)
plt.plot(false_pos_rate, true_pos_rate, linewidth=2, label='AUC = %0.3f'% roc_auc)
plt.plot([1,0],[1,0], linewidth=2)
plt.xlim([-0.01, 1])
plt.ylim([0, 1.01])
plt.legend(loc='lower right')
plt.title('Receiver operating characteristic curve (ROC)')
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()